In [ ]:
from keras.layers import Convolution2D, MaxPooling2D, Conv2DTranspose, Dropout,InputLayer, Flatten, Dense, BatchNormalization
from keras.models import Sequential
import numpy as np
from keras.datasets import mnist
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import np_utils

In [ ]:
np.random.seed(0)

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
def noise(array):
    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
    loc=0.0, scale=1.0, size=array.shape)
    return np.clip(noisy_array, 0.0, 1.0)   

In [ ]:
noisy_X_train = noise(X_train)
noisy_X_test = noise(X_test)

In [ ]:
n = 5  
plt.figure(figsize=(10, 4))
for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_train[i], cmap='gray')
    plt.title("Original")
    plt.axis('off')

    # Noisy
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(noisy_X_train[i], cmap='gray')
    plt.title("Noisy")
    plt.axis('off')

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
noisy_X_train = noisy_X_train.reshape(-1, 28, 28, 1)
noisy_X_test = noisy_X_test.reshape(-1, 28, 28, 1)

In [ ]:
autoencoder = Sequential()

In [ ]:
autoencoder.add(InputLayer(input_shape=(28, 28, 1)))

In [ ]:
autoencoder.add(Convolution2D(32, (3, 3), activation="relu", padding="same"))
autoencoder.add(MaxPooling2D((2, 2), padding="same"))
autoencoder.add(Convolution2D(32, (3, 3), activation="relu", padding="same"))
autoencoder.add(MaxPooling2D((2, 2), padding="same"))

In [ ]:
autoencoder.add(Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"))
autoencoder.add(Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"))
autoencoder.add(Convolution2D(1, (3, 3), activation="sigmoid", padding="same"))

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(noisy_X_train, X_train,
                epochs=5,
                batch_size=128,
                validation_data=(noisy_X_test, X_test))

In [ ]:
clean_images = autoencoder.predict(noisy_X_test)

In [ ]:
n = 10
indices = np.random.randint(0, noisy_X_test.shape[0], size=n)

In [ ]:
plt.figure(figsize=(20, 6))

In [ ]:
for i, idx in enumerate(indices):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(noisy_X_test[idx].reshape(28, 28), cmap='gray')
    plt.title("Noisy")
    plt.axis('off')
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(clean_images[idx].reshape(28, 28), cmap='gray')
    plt.title("Denoised")
    plt.axis('off')

    # Ground truth (original image)
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
imgplot = plt.imshow(X_train[3,:,:],cmap='viridis')
plt.show()
imgplot = plt.imshow(X_train[16,:,:],cmap='viridis')
plt.show()
imgplot = plt.imshow(X_train[1,:,:],cmap='plasma')
plt.show()
imgplot = plt.imshow(X_train[9,:,:],cmap='magma')
plt.show()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
model = Sequential()
model.add(Convolution2D(64, (3, 3), activation='relu', input_shape=(28,28,1), padding="same"))

In [ ]:
print(model.output_shape)
model.add(Convolution2D(64, (3, 3), activation='relu', padding="same"))

In [ ]:
print(model.output_shape)
model.add(Convolution2D(64, (3, 3), activation='relu', padding="same"))

In [ ]:
print(model.output_shape)

Pooling Layer:

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))
print(model.output_shape)

Dropout layer to avoid overfitting

In [ ]:
model.add(Dropout(0.5)) 
# output Fully connected Dense layers:
model.add(Flatten())
print(model.output_shape)

In [ ]:
model.add(Dense(256, activation='relu'))
print(model.output_shape)

In [ ]:
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
print(model.output_shape)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],
              optimizer='RMSprop')
# more info about loss functions: https://keras.io/losses
# more infor about Optimizers: https://keras.io/optimizers

In [ ]:
print(model.summary())
history = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_split=0.2)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b-', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r-', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes = model.predict(X_test)
score = model.evaluate(X_test, y_test, verbose=1)
print('The accuracy is: ', score[1])